In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import json

import pandas as pd

options = Options()

#Comment this out if you want to see the browser, mainly usefull for debugging
options.add_argument("--headless")
options.add_argument("--user-agent=Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.5112.79 Safari/537.36")

pd.set_option('display.max_rows', 1000)

driver = webdriver.Chrome(options=options)

In [7]:
# Accesses Soccer API for PrizePicks and captures all the text
def get_pp_lines():
    driver.get("https://api.prizepicks.com/projections?league_id=82")
    data = driver.find_element(By.TAG_NAME, "pre").text

    # Loads into JSON format
    json_data = json.loads(data)

    players = [{"id": elem["id"], "name": elem["attributes"]["name"], "team": elem["attributes"]["team"], } 
    for elem in json_data["included"] 
    if elem["type"] == "new_player"]

    projections = json_data["data"]
    for player in players:
        player["stats"] = {}
        for projection in projections:
            if projection["type"] == "projection":
                player_id = projection["relationships"]["new_player"]["data"]["id"]

                flash_sale = projection["attributes"]["flash_sale_line_score"]
                line_score = projection["attributes"]["line_score"]
                stat_type = projection["attributes"]["stat_type"]
                against = projection["attributes"]["description"]

                if flash_sale:
                    score = flash_sale
                else:
                    score = line_score
                if player["id"] == player_id:
                    player["opponent"] = against
                    player["stats"][stat_type] = score

    df = pd.DataFrame(players)

    df['bet types'] = df['stats'].apply(lambda x: list(x.keys()))
    df['lines'] = df['stats'].apply(lambda x: list(x.values()))
    df = df.drop(columns=['stats'])

    lines_list_of_lists = df['lines'].tolist()
    lines_list = [item for sublist in lines_list_of_lists for item in sublist]
    df = df.explode('bet types', ignore_index=True)
    df['lines'] = lines_list

    shots_df = df[df['bet types'] == "Shots"]
    sogs_df = df[df['bet types'] == "Shots On Target"]
    shots_df = shots_df.drop(columns=['bet types', 'id'])
    sogs_df = sogs_df.drop(columns=['bet types', 'id'])

    return shots_df, sogs_df

def get_pp_shots():
    return get_pp_lines()[0]

def get_pp_sog():
    return get_pp_lines()[1]

,name,team,opponent,lines
27,Gastón Benedetti,Estudiantes de La Plata,Rosario Central,0.5
36,Ramon Abila,Colon de Santa Fe,River Plate,0.5
53,Salomon Rondon,River Plate,Colon de Santa Fe,0.5
61,Kevin Cabral,Colorado,Portland,0.5
62,Kwadwo Opoku,LAFC,LA Galaxy,0.5
99,Adam Bareiro,San Lorenzo,Racing Club,0.5
109,Lautaro Giaccone,Rosario Central,Estudiantes de La Plata,0.5
124,Gabriel Hauche,Racing Club,San Lorenzo,0.5
126,Ignacio Malcorra,Rosario Central,Estudiantes de La Plata,0.5
140,Jaminton Campaz,Rosario Central,Estudiantes de La Plata,0.5
